In [1]:
from gensim.models import Word2Vec
import networkx as nx
from rdflib import Graph
import random
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error


# 1. 加载 RDF 图数据
def load_rdf_data():
    graphs = []
    try:
        g1 = Graph()
        g1.parse(
            "/Users/yingji/Library/CloudStorage/OneDrive-MicrosoftOffice365/VU_course/AI_master/ML for Graph/RDF2Vec/dataset/AIFB.n3",
            format="n3")
        graphs.append(g1)

        # g2 = Graph()
        # g2.parse(
        #     "/Users/yingji/Library/CloudStorage/OneDrive-MicrosoftOffice365/VU_course/AI_master/ML for Graph/RDF2Vec/dataset/BGS.nt",
        #     format="nt")
        # graphs.append(g2)
        #
        # g3 = Graph()
        # g3.parse(
        #     "/Users/yingji/Library/CloudStorage/OneDrive-MicrosoftOffice365/VU_course/AI_master/ML for Graph/RDF2Vec/dataset/wikidata-20250117-lexeme-BETA.nt",
        #     format="nt")
        # graphs.append(g3)
    except Exception as e:
        print(f"Error parsing RDF files: {e}")
    return graphs


# 2. 使用图遍历生成路径序列 (广度优先搜索)
def generate_graph_walks(graphs, depth=8):
    walks = []
    for graph in graphs:
        nx_graph = nx.DiGraph()
        for subj, pred, obj in graph:
            nx_graph.add_edge(str(subj), str(obj), label=str(pred))  # 记录边的label

        for node in nx_graph.nodes:
            queue = [(node, [node])]  # (当前节点，路径)
            for _ in range(depth):
                next_queue = []
                for current, path in queue:
                    neighbors = list(nx_graph.neighbors(current))
                    for neighbor in neighbors:
                        edge_label = nx_graph[current][neighbor]['label']  # 获取边的label

                        # 将边的label插入路径中
                        new_path = path + [edge_label, neighbor]
                        next_queue.append((neighbor, new_path))

                        # 确保路径长度到达 depth 时才存储
                        if len(new_path) == (2 * depth - 1):  # 节点+边交替，长度为2*depth-1
                            walks.append(new_path)

                queue = next_queue
    return walks


# 3. 使用 Weisfeiler-Lehman 子树算法生成路径
def generate_wl_subtree_walks(graphs, iterations=4, depth=2):
    walks = []
    for graph in graphs:
        nx_graph = nx.DiGraph()
        for subj, pred, obj in graph:
            nx_graph.add_edge(str(subj), str(obj), label=str(pred))

        for node in nx_graph.nodes:
            paths = [[node]]
            for _ in range(iterations):
                new_paths = []
                for path in paths:
                    if len(path) < depth:
                        neighbors = list(nx_graph.neighbors(path[-1]))
                        for neighbor in neighbors:
                            new_paths.append(path + [neighbor])
                paths.extend(new_paths)
            walks.extend(paths)
    return walks


# 4. 使用 CBOW 生成嵌入
def train_word2vec(walks, dimensions=200, window_size=5, sg=0, negative=25, epochs=1):
    model = Word2Vec(
        sentences=walks,
        vector_size=dimensions,
        window=window_size,
        sg=sg,
        negative=negative,
        epochs=epochs,
        workers=4
    )
    for i in range(10):  # 进行 10 轮手动迭代
        model.train(walks, total_examples=len(walks), epochs=1)
    return model


# 3. 使用 Skip-gram 模型生成嵌入
def train_skipgram_model(walks, dimensions=200, window_size=5, sg=1, negative=25, epochs=1):
    model = Word2Vec(
        sentences=walks,
        vector_size=dimensions,
        window=window_size,
        sg=sg,  # 使用 Skip-gram
        negative=negative,
        epochs=epochs,
        workers=4
    )
    for i in range(10):  # 进行 10 轮手动迭代
        model.train(walks, total_examples=len(walks), epochs=1)
    return model


# 5. 特征提取
def extract_features(model, nodes):
    features = {node: model.wv[node] for node in nodes if node in model.wv}
    return features


# 6. 分类/回归任务
def evaluate_features(features, labels, task="classification"):
    X = list(features.values())
    y = [labels.get(node, 0) for node in features.keys()]
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    if task == "classification":
        model = RandomForestClassifier()
    else:
        model = RandomForestRegressor()

    accuracies = []
    rmses = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = [X[i] for i in train_index], [X[i] for i in test_index]
        y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        if task == "classification":
            accuracies.append(accuracy_score(y_test, predictions))
        else:
            rmses.append(mean_squared_error(y_test, predictions, squared=False))

    if task == "classification":
        print("Average Accuracy:", sum(accuracies) / len(accuracies))
    else:
        print("Average RMSE:", sum(rmses) / len(rmses))


In [2]:


graphs = load_rdf_data()
print("RDF 数据加载完成！")

RDF 数据加载完成！


In [3]:
for graph in graphs:
    print(graph)
    break




[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


In [4]:
import random
graph = graphs[0]

# 转换为三元组列表
triples = list(graph)

# 随机采样 100 个三元组
sampled_triples = random.sample(triples, 1500)

# 创建新的图对象，并添加采样的三元组
sampled_graph = Graph()
for triple in sampled_triples:
    sampled_graph.add(triple)

# 不覆盖原始 graphs
sampled_graphs = [sampled_graph]


import random
graph = graphs[0]

# 转换为三元组列表
triples = list(graph)

# 随机采样 100 个三元组
sampled_triples = random.sample(triples, 1500)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
# nodes = [str(node) for graph in graphs for node in graph.all_nodes()]
for train_index, test_index in kf.split(sampled_triples):
    train_graph_list, test_graph_list = [sampled_triples[i] for i in train_index], [sampled_triples[i] for i in test_index]
    train_graph = Graph()

# 添加采样的三元组
    for triple in train_graph_list:
        train_graph.add(triple)

    train_graphs = [train_graph]

    bfs_walks = generate_graph_walks(graphs, depth=8)
    CBOW_embedding_model = train_word2vec(bfs_walks, dimensions=200, window_size=5, sg=0, negative=25)
    print("嵌入模型训练完成！")

    train_nodes = [str(node) for graph in train_graphs for node in graph.all_nodes()]
    features = extract_features(CBOW_embedding_model, train_nodes)



In [5]:


# 生成广度优先路径序列
bfs_walks = generate_graph_walks(sampled_graphs, depth=8)
print("广度优先路径序列生成完成！")

广度优先路径序列生成完成！


In [6]:

# 生成 Weisfeiler-Lehman 子树路径序列
wl_walks = generate_wl_subtree_walks(graphs, iterations=4, depth=2)
print("Weisfeiler-Lehman 子树路径序列生成完成！")

Weisfeiler-Lehman 子树路径序列生成完成！


In [7]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
nodes = [str(node) for graph in graphs for node in graph.all_nodes()]
for train_index, test_index in kf.split(nodes):

        X_train, X_test = [bfs_walks[i] for i in train_index], [bfs_walks[i] for i in test_index]
        bfs_walks = generate_graph_walks(X_train, depth=8)
        CBOW_embedding_model = train_word2vec(bfs_walks, dimensions=200, window_size=5, sg=0, negative=25)
        print("嵌入模型训练完成！")

        nodes = [str(node) for graph in graphs for node in graph.all_nodes()]
        features = extract_features(CBOW_embedding_model, nodes)


嵌入模型训练完成！
嵌入模型训练完成！
嵌入模型训练完成！
嵌入模型训练完成！
嵌入模型训练完成！
嵌入模型训练完成！
嵌入模型训练完成！
嵌入模型训练完成！
嵌入模型训练完成！
嵌入模型训练完成！


In [8]:

# 训练嵌入模型
CBOW_embedding_model = train_word2vec(X_train, dimensions=200, window_size=5, sg=0, negative=25)
print("嵌入模型训练完成！")

嵌入模型训练完成！


In [9]:

# 训练 Skip-gram 嵌入模型
SG_embedding_model = train_skipgram_model(bfs_walks, dimensions=200, window_size=5, sg=1, negative=25)
print("Skip-gram 嵌入模型训练完成！")

Skip-gram 嵌入模型训练完成！


In [10]:

# 提取嵌入特征
nodes = [str(node) for graph in graphs for node in graph.all_nodes()]
features = extract_features(SG_embedding_model, nodes)
print("嵌入特征提取完成！")

嵌入特征提取完成！


In [11]:

# 评估分类任务
labels = {"node1": 0, "node2": 1}  # 替换为你的节点标签
evaluate_features(features, labels, task="classification")


Average Accuracy: 1.0
